In [73]:
#import libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [74]:


#create useable data set for X

dataX = np.zeros((2225,9635))
dataY = np.zeros((2225,1))

docs_file = open('bbc.docs','r')
docs_content = docs_file.read()
docs_content = docs_content.split('\n')
#read file
with open('bbc.mtx','r') as file:

    file.readline()
    file.readline()

    for line in file:
        words = line.split()

        term = int(words[0])
        article = int(words[1])
        frequency= float(words[2])
        if frequency>0:
            dataX[article-1][term-1] = 1
#fill DataY
for i in range(len(docs_content)):
    article_type = docs_content[i].split('.')[0]
    article_type = article_type.lower()
    if article_type == 'business':
        dataY[i] = 0

    elif article_type == 'entertainment':
        dataY[i] = 1

    elif article_type == 'politics':
        dataY[i] = 2

    elif article_type == 'sport':
        dataY[i] = 3

    elif article_type == 'tech':
        dataY[i] = 4

    else:
        if article_type == '':
            pass
        else:
            raise ValueError("Invalid article type: {}".format(article_type))





In [75]:
#split the data into training adn testing data
X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=0.2, random_state=42)

print ('X_train.shape: ', X_train.shape, "y_train.shape:",y_train.shape, "X_test.shape:",X_test.shape, "y_test.shape",y_test.shape)


X_train.shape:  (1780, 9635) y_train.shape: (1780, 1) X_test.shape: (445, 9635) y_test.shape (445, 1)


In [76]:
#calculate the priors
num_of_each_type_doc= np.array([0,0,0,0,0])

for y in y_train:
    num_of_each_type_doc[int(y[0])] +=1

priors = num_of_each_type_doc/len(y_train)

print("priors:",priors)

priors: [0.22191011 0.17640449 0.19157303 0.22977528 0.18033708]


In [77]:


#calculate the probabilities P(x|Ci)

p_X_given_Ci_Matrix = np.zeros((5,9635))

for i in range(len(X_train)):
    article_type = int(y_train[i][0])
    p_X_given_Ci_Matrix[article_type] += X_train[i]

for i in range(5):
    for j in range(9635):
        p_X_given_Ci_Matrix[i][j] = (p_X_given_Ci_Matrix[i][j] + 1)/(num_of_each_type_doc[i]+2)
print (p_X_given_Ci_Matrix)
for row in p_X_given_Ci_Matrix:
    for col in row:
        if col == 0:
            raise ValueError



[[0.25944584 0.31234257 0.19143577 ... 0.00251889 0.00251889 0.00251889]
 [0.22151899 0.17405063 0.03481013 ... 0.00316456 0.00316456 0.00316456]
 [0.36151603 0.02915452 0.06413994 ... 0.00291545 0.00291545 0.00291545]
 [0.33819951 0.05109489 0.04379562 ... 0.00243309 0.00243309 0.00243309]
 [0.28173375 0.17337461 0.05263158 ... 0.0123839  0.0123839  0.00619195]]


In [78]:
#predict the test_set
y_pred = np.array([])
x_count = 0
for x in X_test:
    print("iter",x_count)
    probs = np.copy(priors)
    
    for i in range(probs.shape[0]):
        for j in range(len(x)):
            if x[j] == 1:
                probs[i] *= p_X_given_Ci_Matrix[i][j]

            elif x[j] == 0:
                probs[i] *= (1 - p_X_given_Ci_Matrix[i][j])

            else:
                raise ValueError
    print(probs)
    y_pred = np.append(y_pred,np.argmax(probs))
    x_count+= 1

print("Accuracy on test set",accuracy_score(y_test,y_pred))




iter 0
[1.36158696e-196 6.20389409e-248 2.91307417e-236 9.59363320e-258
 5.61076686e-251]
iter 1
[8.05769985e-207 6.70142957e-257 1.25137616e-236 2.04777890e-251
 1.59545360e-248]
iter 2
[0.00000000e+000 0.00000000e+000 0.00000000e+000 1.98936867e-261
 0.00000000e+000]
iter 3
[2.07390875e-183 4.22126051e-230 9.75522253e-233 1.00604569e-243
 3.18694580e-228]
iter 4
[1.24426956e-267 6.08905107e-283 2.76640760e-234 1.02669107e-307
 1.45645192e-261]
iter 5
[1.06067894e-279 6.96535922e-283 9.67872482e-275 1.13521124e-236
 2.30390135e-286]
iter 6
[1.15716731e-262 3.25058795e-261 1.29391131e-270 3.90971380e-213
 2.14259204e-274]
iter 7
[1.04764385e-219 2.00265822e-234 2.35797725e-213 1.86322721e-246
 1.73090123e-236]
iter 8
[0.0e+000 1.5e-322 0.0e+000 0.0e+000 0.0e+000]
iter 9
[4.61385280e-265 2.44637941e-270 1.03214901e-254 3.59266697e-285
 1.78994167e-279]
iter 10
[2.99234908e-261 6.42285340e-323 7.88677129e-307 0.00000000e+000
 1.41179258e-319]
iter 11
[5.50891013e-231 3.45791276e-169 8.21

In [79]:
#predict the train_set
y_pred = np.array([])
x_count = 0
for x in X_train:
    print("iter",x_count)
    probs = np.copy(priors)
    
    for i in range(probs.shape[0]):
        for j in range(len(x)):
            if x[j] == 1:
                probs[i] *= p_X_given_Ci_Matrix[i][j]

            elif x[j] == 0:
                probs[i] *= (1 - p_X_given_Ci_Matrix[i][j])

            else:
                raise ValueError
    print(probs)
    y_pred = np.append(y_pred,np.argmax(probs))
    x_count+= 1

print("Accuracy on test set",accuracy_score(y_train,y_pred))

iter 0
[1.39493465e-214 4.16255863e-205 7.80870818e-217 5.13960815e-165
 4.24540628e-222]
iter 1
[3.72862373e-298 1.31228459e-278 1.13831809e-309 2.74160485e-318
 1.28874982e-255]
iter 2
[7.63177789e-274 3.96650533e-265 6.27489006e-258 1.31819040e-226
 1.93262541e-275]
iter 3
[2.23536504e-258 3.54578049e-277 1.34999750e-264 3.46884666e-295
 9.63638438e-234]
iter 4
[9.63591049e-292 1.22841785e-236 2.30795295e-299 1.06588322e-293
 9.86861930e-292]
iter 5
[1.47452539e-192 1.55773392e-255 1.55560736e-248 3.55614147e-267
 7.25784832e-244]
iter 6
[5.05401212e-268 4.09507737e-277 2.24333718e-266 6.07169320e-282
 1.38174932e-236]
iter 7
[4.16861867e-300 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000]
iter 8
[1.43549071e-315 1.38005995e-232 3.50685391e-302 1.61821361e-301
 1.66003784e-303]
iter 9
[1.01689553e-195 3.82006150e-205 2.36211283e-171 7.25853059e-207
 1.88696294e-211]
iter 10
[8.94942545e-258 4.20042239e-251 2.42938171e-215 6.26431769e-260
 3.37224993e-269]
iter 11
[